# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data

# Order of play

Part 1: Data Curation  
Part 2: Data Pre-processing  
Part 3: Evaluation, Baselines, Traditional ML  
Part 4: Deep Learning and LLMs  
Part 5: Fine-tuning a Frontier Model  

## Part 3: Evaluation, Baselines, Traditional ML

Today we'll write some simple models to predict the price of a product

We'll use an approach to evaluate the performance of the model

And we'll test some Baseline Models using Traditional machine learning

In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from pricer.evaluator import evaluate
from pricer.items import Item

In [2]:
LITE_MODE = False

In [ ]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

In [4]:
def random_pricer(item):
    return random.randrange(1,1000)

In [5]:
random.seed(42)
evaluate(random_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$436 $1 $29 $690 $252 $21 $85 $72 $719 $225 $20 $380 $894 $505 $11 $572 $354 $17 $179 $23 $90 $115 $433 $442 $304 $122 $291 $714 $567 $639 $539 $370 $66 $380 $489 $534 $769 $835 $207 $740 $626 $84 $680 $178 $129 $260 $142 $189 $836 $580 $310 $25 $380 $270 $47 $234 $861 $313 $417 $259 $591 $33 $657 $361 $79 $38 $757 $500 $263 $5 $534 $284 $570 $625 $584 $871 $759 $361 $575 $178 $602 $60 $17 $579 $207 $732 $115 $224 $756 $193 $866 $9 $370 $250 $456 $423 $821 $217 $103 $195 $264 $98 $650 $135 $470 $842 $675 $264 $43 $325 $591 $138 $516 $619 $56 $2 $449 $369 $221 $845 $640 $616 $501 $59 $502 $273 $844 $688 $616 $81 $164 $705 $52 $795 $259 $396 $70 $2 $89 $798 $902 $331 $818 $716 $129 $186 $627 $2 $141 $873 $918 $553 $423 $7 $253 $136 $204 $707 $255 $502 $19 $739 $557 $426 $80 $575 $39 $321 $185 $132 $497 $484 $326 $751 $53 $733 $85 $64 $549 $71 $158 $672 $133 $362 $16 $373 $258 $544 $420 $515 $223 $944 $532 $743 $866 $68 $527 $459 $67 $673 

In [6]:
# That was fun!
# We can do better - here's another rather trivial model

training_prices = [item.price for item in train]
training_average = sum(training_prices) / len(training_prices)
print(training_average)

def constant_pricer(item):
    return training_average

140.56967544948907


In [7]:
evaluate(constant_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$78 $25 $86 $71 $111 $89 $4 $75 $105 $189 $572 $238 $121 $86 $61 $108 $61 $91 $70 $22 $7 $17 $56 $34 $191 $312 $354 $121 $42 $61 $121 $81 $19 $60 $25 $678 $81 $85 $73 $103 $59 $61 $106 $114 $79 $116 $123 $109 $5 $61 $105 $11 $334 $21 $87 $6 $134 $101 $62 $129 $95 $63 $50 $31 $488 $51 $99 $304 $16 $65 $109 $124 $139 $122 $91 $105 $16 $131 $124 $122 $21 $129 $111 $42 $114 $81 $42 $165 $21 $95 $119 $46 $121 $106 $132 $88 $107 $17 $129 $434 $41 $24 $104 $2 $108 $23 $116 $259 $110 $158 $81 $174 $110 $12 $55 $29 $116 $121 $85 $38 $125 $52 $70 $25 $59 $81 $121 $42 $38 $2 $69 $3 $55 $111 $76 $126 $64 $71 $12 $2 $76 $109 $60 $121 $53 $109 $96 $369 $124 $108 $122 $35 $94 $1 $121 $138 $92 $85 $179 $91 $148 $115 $99 $128 $699 $118 $307 $91 $101 $131 $116 $119 $279 $118 $39 $8 $113 $48 $46 $48 $514 $116 $159 $108 $91 $119 $7 $74 $81 $114 $106 $90 $106 $2 $41 $61 $29 $140 $90 $115 

In [8]:
def get_features(item):
    return {
        "weight": item.weight,
        "weight_unknown": 1 if item.weight==0 else 0,
        "text_length": len(item.summary)
    }

In [9]:
def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test)

In [10]:
# Traditional Linear Regression!

np.random.seed(42)

# Separate features and target
feature_columns = ['weight', 'weight_unknown', 'text_length']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

# Train a Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(feature_columns, model.coef_):
    print(f"{feature}: {coef}")
print(f"Intercept: {model.intercept_}")

# Predict the test set and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

weight: 0.4486789805756444
weight_unknown: -6.6279988778273475
text_length: 0.24694518955631115
Intercept: 51.11099822543963
Mean Squared Error: 25615.84348857261
R-squared Score: -0.05946814914319187


In [11]:
def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [12]:
evaluate(linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$55 $29 $87 $90 $92 $60 $1 $78 $104 $192 $556 $228 $141 $81 $50 $124 $69 $83 $50 $8 $10 $9 $61 $2 $175 $298 $345 $114 $34 $55 $100 $77 $10 $38 $24 $676 $78 $94 $50 $90 $48 $55 $83 $102 $82 $119 $100 $106 $2 $52 $106 $9 $353 $28 $92 $26 $136 $110 $37 $121 $97 $69 $43 $17 $460 $41 $83 $288 $4 $86 $109 $101 $139 $113 $99 $103 $8 $122 $122 $111 $25 $107 $98 $10 $108 $63 $60 $164 $33 $82 $109 $41 $101 $94 $114 $104 $97 $9 $151 $426 $37 $11 $135 $5 $100 $25 $107 $277 $102 $182 $90 $169 $135 $11 $45 $38 $100 $119 $98 $21 $125 $72 $76 $32 $35 $60 $109 $61 $10 $2 $78 $12 $58 $95 $58 $106 $65 $65 $1 $8 $61 $110 $54 $107 $33 $91 $93 $364 $107 $92 $118 $44 $76 $19 $111 $122 $102 $61 $138 $76 $135 $116 $83 $113 $710 $120 $269 $101 $107 $125 $110 $124 $275 $101 $52 $4 $119 $26 $48 $53 $481 $141 $139 $64 $102 $116 $19 $77 $78 $103 $94 $99 $122 $1 $30 $65 $25 $133 $112 $113 

In [13]:
prices = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [14]:
np.random.seed(42)
vectorizer = CountVectorizer(max_features=2000, stop_words='english')
X = vectorizer.fit_transform(documents)


In [15]:
# Here are the 1,000 most common words that it picked, not including "stop words":

selected_words = vectorizer.get_feature_names_out()
print(f"Number of selected words: {len(selected_words)}")
print("Selected words:", selected_words[1000:1020])

Number of selected words: 2000
Selected words: ['jack' 'jacket' 'jeep' 'jet' 'jigsaw' 'joint' 'joints' 'kawasaki'
 'keeping' 'keeps' 'key' 'keyboard' 'keypad' 'keys' 'kg' 'khz' 'kia'
 'kickstand' 'kids' 'king']


In [16]:
regressor = LinearRegression()
regressor.fit(X, prices)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [17]:
def natural_language_linear_regression_pricer(item):
    x = vectorizer.transform([item.summary])
    return max(regressor.predict(x)[0], 0)

In [18]:
evaluate(natural_language_linear_regression_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$67 $124 $55 $6 $179 $211 $52 $50 $73 $7 $535 $186 $101 $150 $67 $96 $58 $49 $37 $29 $15 $87 $28 $9 $274 $241 $141 $13 $56 $80 $78 $57 $10 $43 $113 $418 $15 $65 $140 $71 $156 $61 $69 $26 $94 $59 $39 $49 $4 $13 $41 $91 $156 $31 $80 $69 $81 $166 $24 $12 $44 $18 $55 $24 $422 $93 $3 $312 $62 $212 $19 $33 $11 $130 $1 $36 $115 $36 $14 $130 $85 $59 $43 $42 $36 $117 $42 $105 $31 $177 $4 $106 $80 $27 $38 $91 $61 $1 $162 $102 $49 $56 $37 $15 $29 $4 $43 $214 $31 $98 $20 $20 $171 $42 $6 $149 $112 $25 $24 $40 $11 $108 $87 $28 $80 $27 $32 $127 $86 $29 $74 $40 $141 $23 $70 $55 $39 $103 $126 $76 $34 $171 $78 $95 $68 $91 $25 $235 $63 $32 $19 $189 $24 $27 $62 $49 $124 $28 $7 $9 $76 $26 $42 $13 $470 $22 $76 $50 $34 $9 $3 $23 $279 $27 $41 $144 $10 $40 $45 $144 $372 $18 $20 $13 $158 $32 $44 $16 $103 $36 $29 $122 $9 $27 $43 $60 $53 $6 $48 $18 

In [ ]:
subset = 15_000
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=4)
rf_model.fit(X[:subset], prices[:subset])

## Random Forest model

The Random Forest is a type of "**ensemble**" algorithm, meaning that it combines many smaller algorithms to make better predictions.

It uses a very simple kind of machine learning algorithm called a **decision tree**. A decision tree makes predictions by examining the values of features in the input. Like a flow chart with IF statements. Decision trees are very quick and simple, but they tend to overfit.

In our case, the "features" are the elements of the Vector - in other words, it's the number of times that a particular word appears in the product description.

So you can think of it something like this:

**Decision Tree**  
\- IF the word "TV" appears more than 3 times THEN  
-- IF the word "LED" appears more than 2 times THEN  
--- IF the word "HD" appears at least once THEN  
---- Price = $500


With Random Forest, multiple decision trees are created. Each one is trained with a different random subset of the data, and a different random subset of the features. You can see above that we specify 100 trees, which is the default.

Then the Random Forest model simply takes the average of all its trees to product the final result.

In [ ]:
def random_forest(item):
    x = vectorizer.transform([item.summary])
    return max(0, rf_model.predict(x)[0])

In [ ]:
evaluate(random_forest, test)

In [ ]:
# This is how to save the model if you want to, particularly if you run this on a larger dataset

# import joblib
# joblib.dump(rf_model, "random_forest.joblib")

## Introducing XGBoost

Like Random Forest, XGBoost is also an ensemble model that combines multiple decision trees.

But unlike Random Forest, XGBoost builds one tree after another, with each next tree correcting for errors in the prior trees, using 'gradient descent'.

It's much faster than Random Forest, so we can run it for the full dataset, and it's typically better at generalizing.

**If this import doesn't work, please skip this! It's not required. On a Mac, you might need to do `brew install libomp` in the terminal.**

In [ ]:
import xgboost as xgb

In [ ]:
np.random.seed(42)

xgb_model = xgb.XGBRegressor(n_estimators=1000, random_state=42, n_jobs=4, learning_rate=0.1)
xgb_model.fit(X, prices)

In [ ]:
def xg_boost(item):
    x = vectorizer.transform([item.summary])
    return max(0, xgb_model.predict(x)[0])

In [ ]:
evaluate(xg_boost, test)